In [141]:
import pandas as pd
import seaborn as sns
import numpy as np

## SUSB Tables
SUSB is an annual series that provides national and subnational data on the distribution of economic data by establishment industry & enterprise size.

https://www.census.gov/programs-surveys/susb.html

The county information is not in easily extractable tables, which makes the data aggregation irritating. 

In [125]:
susb = pd.DataFrame()

rename_map = {'COUNTY_DESCRIPTION':'County_Name',
              'NUMBER_OF_FIRMS':'Firms',
              'NUMBER_OF_ESTABLISHMENTS':'Establishments',
              'EMPLOYMENT':'Employment',
              'ANNUAL_PAYROLL_($1,000)':'Payroll_$1000',
              'Annual_Payroll\n($1,000)':'Payroll_$1000',
              'Annual_Payroll_($1,000)':'Payroll_$1000',
              'ENTERPRISE_EMPLOYMENT_SIZE':'Enterprise_Size',
              'STATE_DESCRIPTION':'State_Name',
              'FIPS_COUNTY_CODE':'FIPS_County',
              'FIPS_COUNTY_CODE_':'FIPS_County',
              'County':'FIPS_County'
             }

urls = ['https://www2.census.gov/programs-surveys/susb/tables/2010/county_totals_2010.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2011/county_totals_2011.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2012/county_totals_2012.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2013/county_totals_2013.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2014/county_totals_2014.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2015/county_totals_2015.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2016/county_totals_2016.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2017/county_totals_2017.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2018/county_3digitnaics_2018.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2019/county_3digitnaics_2019.xlsx'\
       ]

urls2 = ['https://www2.census.gov/programs-surveys/susb/tables/2010/county_totals_2010.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2011/county_totals_2011.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2012/county_totals_2012.xls',
        'https://www2.census.gov/programs-surveys/susb/tables/2013/county_totals_2013.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2014/county_totals_2014.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2015/county_totals_2015.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2016/county_totals_2016.xlsx',
        'https://www2.census.gov/programs-surveys/susb/tables/2017/county_totals_2017.xlsx']
j = 2010

def get_skiprows(n):
    """
    Likely not the best way to do this, but returns the right skiprows for each dataset"""
    if n < 2012:
        return [0,1,2,3,5,6]
    elif n == 2012:
        return [0,1,2,3,4,6,7]
    elif n == 2013:
        return [0,1,2,3,5,6]
    elif n < 2016:
        return [0,1,2,3,4,6,7]
    elif n < 2017:
        return [0,1,2,3,4,5,7,8]
    elif n < 2018:
        return 2
    else:
        return 2
for i in urls:
    df = pd.read_excel(i,skiprows=get_skiprows(j)) # bad excel tables need custom skipped row
    df.columns = df.columns.str.strip() # remove leading/tailing whitespace
    df.columns = df.columns.str.replace(r'\s+','_',regex=True) # regex removes multiple spaces with _
    df = df.rename(rename_map,axis=1)
    df.Enterprise_Size = df.Enterprise_Size.replace(r'\s+', ' ', regex=True)
    df = df.replace(dict.fromkeys(['1: Total','01: Total'], ['Total']))
    df = df.replace(['VIRGINIA'], ['Virginia'])
    if 'NAICS_Description' in df.columns:
        df = df.query('State_Name=="Virginia"&Enterprise_Size=="Total"&NAICS_Description=="Total"')\
               .loc[:,['FIPS_County','County_Name','Firms','Establishments','Employment','Payroll_$1000']]
    else:
        df = df.query('State_Name=="Virginia"&Enterprise_Size=="Total"')\
               .loc[:,['FIPS_County','County_Name','Firms','Establishments','Employment','Payroll_$1000']]
    df['year']=j
    df.County_Name = df.County_Name.str.lower()
    j+=1
    susb = pd.concat([susb, df])

In [127]:
susb.query('County_Name=="bedford"')

,FIPS_County,County_Name,Firms,Establishments,Employment,Payroll_$1000,year
14237,19,bedford,1390,1436,12682,387881,2010
14243,19,bedford,1317,1366,12049,415031,2011
14251,19,bedford,1299,1344,11881,416854,2012
14266,19,bedford,1306,1353,11923,426731,2013
14260,19,bedford,1309,1357,11809,439816,2014
14260,19,bedford,1326,1376,11737,443244,2015
14266,19,bedford,1327,1383,11768,453105,2016
14007,19,bedford,1606,1715,15567,595647,2017
449758,19,bedford,1616,1726,16305,621971,2018
449364,19,bedford,1624,1736,17799,717521,2019


In [121]:
len(susb.FIPS_County.unique())

135

In [7]:
susb.dtypes

FIPS_County        int64
County_Name       object
Firms              int64
Establishments     int64
Employment         int64
Payroll_$1000      int64
year               int64
dtype: object

## Small Area Poverty Income Estimates
https://www.census.gov/programs-surveys/saipe.html

These tables use a regression to estimate poverty percentage per county with 90% intervals. I haven't gone into the regression estimates. 

In [8]:
pov_tot = pd.DataFrame()

for i in range(10,20):
    url = 'https://www2.census.gov/programs-surveys/saipe/datasets/20'+str(i)+'/20'+str(i)+'-state-and-county/est'+str(i)+'-va.txt'
    pov = pd.read_fwf(url, usecols=[1,5,20,29],names = ['FIPS_County','Percent_in_Poverty','Median_Income','County_Name'])
    pov['year'] = '20'+str(i)
    pov_tot = pd.concat([pov_tot, pov])

In [122]:
pov_tot.Percent_in_Poverty = pov_tot.Percent_in_Poverty.astype(float)
pov_tot.year = pov_tot.year.astype(int)
pov_tot.County_Name = pov_tot.County_Name.str.lower()

In [10]:
len(pov_tot.FIPS_County.unique())

135

In [11]:
pov_tot.dtypes

FIPS_County             int64
Percent_in_Poverty    float64
Median_Income           int64
County_Name            object
year                    int32
dtype: object

In [12]:
pov_tot

,FIPS_County,Percent_in_Poverty,Median_Income,County_Name,year
0,0,11.1,60665,Virginia,2010
1,1,20.5,37312,Accomack County,2010
2,3,9.1,61845,Albemarle County,2010
3,5,13.1,43110,Alleghany County,2010
4,7,11.4,49057,Amelia County,2010
...,...,...,...,...,...
129,800,9.8,77847,Suffolk city,2019
130,810,7.3,78491,Virginia Beach city,2019
131,820,12.3,44619,Waynesboro city,2019
132,830,18.2,56569,Williamsburg city,2019


## VA Economic Indicators Data

https://www.bea.gov/data/gdp/gdp-county-metro-and-other-areas
A comprehensive measure of the economies of counties, metropolitan statistical areas, and some other local areas. Gross domestic product estimates the value of the goods and services produced in an area. It can be used to compare the size and growth of county economies across the nation. The table we used is specifically personal income by major component and earnings by industry per county and metropolitan area.

Is there anyway to get this data with the FIPS code? Would make the merge much easier
Brian: GeoFIPS is the FIPS code but I don't know how to manipulate

In [123]:
econ = pd.read_csv('va_economic_indicators.csv')

In [202]:
econ = econ.rename({'GeoName':'County_Name'},axis=1)
econ_year = econ.loc[(econ['year'] > 2009) & (econ['year'] < 2020)]
econ_year.County_Name = econ.County_Name.str.split(',').str[0]

econreplace = {
    'Accomack': 'accomack county',
    'Albemarle + Charlottesville': 'albemarle county',
    'Alleghany + Covington': 'alleghany county',
    'Amelia': 'amelia county',
    'Amherst': 'amherst county',
    'Appomattox': 'appomattox county',
    'Arlington': 'arlington county',
    'Augusta': 'augusta county',
    'Bath': 'bath county',
    'Bedford': 'bedford county',
    'Bland': 'bland county',
    'Botetourt': 'botetourt county',
    'Brunswick': 'brunswick county',
    'Buchanan': 'buchanan county',
    'Buckingham': 'buckingham county',
    'Campbell + Lynchburg': 'campbell county',
    'Caroline': 'caroline county',
    'Carroll + Galax': 'carroll county',
    'Charles City': 'charles city county',
    'Charlotte': 'charlotte county',
    'Chesterfield': 'chesterfield county',
    'Clarke': 'clarke county',
    'Craig': 'craig county',
    'Culpeper': 'culpeper county',
    'Cumberland': 'cumberland county',
    'Dickenson': 'dickenson county',
    'Dinwiddie': 'dinwiddie county',
    'Essex': 'essex county',
    'Fairfax': 'fairfax county',
    'Fauquier': 'fauquier county',
    'Floyd': 'floyd county',
    'Fluvanna': 'fluvanna county',
    'Franklin': 'franklin county',
    'Frederick + Winchester': 'frederick county',
    'Giles': 'giles county',
    'Gloucester': 'gloucester county',
    'Goochland': 'goochland county',
    'Grayson': 'grayson county',
    'Greene': 'greene county',
    'Greensville + Emporia': 'greensville county',
    'Halifax': 'halifax county',
    'Hanover': 'hanover county',
    'Henrico': 'henrico county',
    'Henry + Martinsville': 'henry county',
    'Highland': 'highland county',
    'Isle of Wight': 'isle of wight county',
    'James City + Williamsburg': 'james city county',
    'King George': 'king and queen county',
    'King William': 'king george county',
    'King and Queen': 'king william county',
    'Lancaster': 'lancaster county',
    'Lee': 'lee county',
    'Loudoun': 'loudoun county',
    'Louisa': 'louisa county',
    'Lunenburg': 'lunenburg county',
    'Madison': 'madison county',
    'Mathews': 'mathews county',
    'Mecklenburg': 'mecklenburg county',
    'Middlesex': 'middlesex county',
    'Montgomery + Radford': 'montgomery county',
    'Nelson': 'nelson county',
    'New Kent': 'new kent county',
    'Northampton': 'northampton county',
    'Northumberland': 'northumberland county',
    'Nottoway': 'nottoway county',
    'Orange': 'orange county',
    'Page': 'page county',
    'Patrick': 'patrick county',
    'Pittsylvania + Danville': 'pittsylvania county',
    'Powhatan': 'powhatan county',
    'Prince Edward': 'prince edward county',
    'Prince George + Hopewell': 'prince george county',
    'Prince William': 'prince william county',
    'Pulaski': 'pulaski county',
    'Rappahannock': 'rappahannock county',
    'Richmond': 'richmond county',
    'Roanoke + Salem': 'roanoke county',
    'Rockbridge': 'rockbridge county',
    'Rockingham + Harrisonburg': 'rockingham county',
    'Russell': 'russell county',
    'Scott': 'scott county',
    'Shenandoah': 'shenandoah county',
    'Smyth': 'smyth county',
    'Southampton + Franklin': 'southampton county',
    'Spotsylvania + Fredericksburg': 'spotsylvania county',
    'Stafford': 'stafford county',
    'Surry': 'surry county',
    'Sussex': 'sussex county',
    'Tazewell': 'tazewell county',
    'Warren': 'warren county',
    'Washington + Bristol': 'washington county',
    'Westmoreland': 'westmoreland county',
    'Wise + Norton': 'wise county',
    'Wythe': 'wythe county',
    'York + Poquoson': 'york county',
    'Alexandria (Independent City)':'alexandria city', 
    'Chesapeake (Independent City)':'chesapeake city',
    'Hampton (Independent City)':'hampton city', 
    'Newport News (Independent City)':'newport news city',
    'Norfolk (Independent City)':'norfolk city', 
    'Portsmouth (Independent City)':'portsmouth city',
    'Richmond (Independent City)':'richmond city', 
    'Roanoke (Independent City)':'roanoke city',
    'Suffolk (Independent City)':'suffolk city', 
    'Virginia Beach (Independent City)':'virginia beach city'
}



C:\Users\brigg\AppData\Local\Temp\ipykernel_21136\1407687528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econ_year.County_Name = econ.County_Name.str.split(',').str[0]


In [203]:
econ_year = econ_year.replace(econreplace)
econ_year = econ_year.reset_index()
econ_year = econ_year.drop(['index','Unnamed: 0'],axis=1)

We lose some counties/cities compared to the previous data sets. I'm not sure if there is anything we can do since some cities are grouped with counties. 

In [204]:
df = pd.merge(pov_tot, susb, how='inner', on=['FIPS_County','year'])

In [205]:
df = df.drop('County_Name_y',axis=1).rename({'County_Name_x':'County_Name'},axis=1)

In [206]:
df

,FIPS_County,Percent_in_Poverty,Median_Income,County_Name,year,Firms,Establishments,Employment,Payroll_$1000
0,1,20.5,37312,accomack county,2010,734,826,9443,256121
1,3,9.1,61845,albemarle county,2010,2297,2514,36318,1611944
2,5,13.1,43110,alleghany county,2010,238,253,2365,64972
3,7,11.4,49057,amelia county,2010,280,286,1866,52307
4,9,13.6,42063,amherst county,2010,567,601,6697,193779
...,...,...,...,...,...,...,...,...,...
1329,800,9.8,77847,suffolk city,2019,1462,1684,25065,1010127
1330,810,7.3,78491,virginia beach city,2019,9538,11316,160538,6931230
1331,820,12.3,44619,waynesboro city,2019,569,611,9103,336617
1332,830,18.2,56569,williamsburg city,2019,456,524,9416,292213


In [207]:
df2 = pd.merge(df, econ_year, how='inner', on=['County_Name','year'])

In [208]:
df3 = df2.reset_index()

In [210]:
df3.isnull().sum()

index                                                                       0
FIPS_County                                                                 0
Percent_in_Poverty                                                          0
Median_Income                                                               0
County_Name                                                                 0
year                                                                        0
Firms                                                                       0
Establishments                                                              0
Employment                                                                  0
Payroll_$1000                                                               0
Earnings by place of work                                                   0
Employee and self-employed contributions for government social insurance    0
Employer contributions for employee pension and insurance funds 

In [191]:
pd.DataFrame.to_csv(df2, '2010_2019_cleaned_data.csv')